In [ ]:
import array, datetime, json, math, os, random

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def YearMonthDayToEpoch(year, month, day):
    return (datetime.datetime(int(year), int(month), int(day)) - datetime.datetime(1970, 1, 1)).total_seconds()

In [ ]:
if not os.path.exists("../data/making-a-point-map/"):
    os.makedirs("../data/making-a-point-map/")

In [ ]:
!wget --no-check-certificate https://eerscmap.usgs.gov/uswtdb/assets/data/uswtdbGeoJSON.zip -P ../data/making-a-point-map/

In [ ]:
!unzip ../data/making-a-point-map/uswtdbGeoJSON.zip -d ../data/making-a-point-map/

In [ ]:
with open("../data/making-a-point-map/uswtdb_v1_1_20180710.geojson") as f:
    data = json.load(f)

In [ ]:
len(data["features"])

In [ ]:
data["features"][0]

In [ ]:
min_year = 2018
max_year = 0
points = []
for feature in data["features"]:
    geometry = feature['geometry']
    properties = feature['properties']
    if properties['p_year'] != -9999:
        lon = geometry['coordinates'][0]
        lat = geometry['coordinates'][1] 
        x,y = LonLatToPixelXY([lon,lat])
        year = properties['p_year']
        month = random.randrange(1,12,1) # bogus month
        day = random.randrange(1,28,1) # bogus day
        epochtime = YearMonthDayToEpoch(year, month, day)
        points += [x,y,epochtime]
        if year < min_year:
            min_year = year
        if year > max_year:
            max_year = year

f.close()
array.array('f', points).tofile(open('../data/making-a-point-map/points.bin', 'wb'))
    

In [ ]:
min_year, max_year

In [ ]:
!rsync -rcav ../data/making-a-point-map timemachine2:/usr2/SUBDOMAIN.earthtime.org/app/misc/

In [ ]:
print "TRUE\tmaking_a_point_map\tMaking EarthTime Maps\tPoints\tUSGS\t1981\t2018\t1\thttps://SUBDOMAIN.earthtime.org/misc/making-a-point-map/points.bin\t\t\tpoint\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tWebGLVectorTile2.prototype._loadData\tWebGLVectorTile2.prototype._setBufferData \t\t3\tWebGLVectorTile2.vectorPointTileVertexShader\tWebGLVectorTile2.vectorPointTileFragmentShader\tWebGLVectorTile2.prototype._drawPoints"